# 01_scrap

## Getting data from The Bangkok Insight

In [1]:
from bs4 import BeautifulSoup
import requests
import datetime
import pandas as pd
import json

In [2]:
link = 'https://www.thebangkokinsight.com/page/'
page = 1

In [3]:
bs = []
while page <= 3:
    r = requests.get(link + str(page) + '/?s=ส่งออกทุเรียน')
    s = BeautifulSoup(r.content, 'html.parser')
    bs.append(s)
    page += 1
r

<Response [200]>

In [4]:
title = []
content = []
source_url = []
publish_date = []
retrieve_date = []

In [5]:
for i in range(len(bs)):
    for j in bs[i].find_all('h3', {'class':'post__title typescale-2'}):
        topic = j.find('a').text.strip()
        title.append(topic)
        source_url.append(j.find('a')['href'])
        
    for t in bs[i].find_all('div', {'class':'post__meta'}):
        publish_date.append(t.find('time')['datetime'].split('G')[0])
        retrieve_date.append(str(datetime.date.today()))

In [6]:
for url in source_url:
    re = requests.get(url)
    soup = BeautifulSoup(re.content, 'html.parser')
    
    for c in soup.find_all('div', {'class':'single-body entry-content typography-copy'}):
        con = c.text.strip()
        con = con.split('อ่านข่าวเพิ่มเติม')[0]
        con = con.replace('\xa0', ' ')
        content.append(con)

In [8]:
df = pd.DataFrame([title, content, source_url, publish_date, retrieve_date]).transpose()
df.columns = ['title', 'content', 'source_url', 'publish_date', 'retrieve_date']
df.head()

,title,content,source_url,publish_date,retrieve_date
0,ข่าวดีชาวสวน จีนเปิดครบทุกด่านแล้ว ส่งออกทุเรี...,“อลงกรณ์” เผยข่าวดี จีนเปิดทุกด่านครบแล้ววันนี...,https://www.thebangkokinsight.com/news/busines...,2022-04-29,2022-11-15
1,ไทยส่งออกทุเรียนทุบสถิติครั้งใหม่ มูลค่าแซงหน้...,ไทยเป็นผู้ส่งออกทุเรียนสดอันดับ 1 ของโลก ล่าสุ...,https://www.thebangkokinsight.com/news/busines...,2021-06-25,2022-11-15
2,เร่งขึ้นทะเบียน ‘GAP’ ยกระดับ ‘สวนทุเรียน’ ผนึ...,เร่งขึ้นทะเบียน ‘GAP’ ยกระดับ ‘สวนทุเรียน’ ผนึ...,https://www.thebangkokinsight.com/news/busines...,2022-11-13,2022-11-15
3,ใช้ ‘GAP Monkey Free Plus’ แก้ปัญหา ‘ถูกกีดกัน...,กรมวิชาการเกษตร ใช้ ‘GAP Monkey Free Plus’ แก้...,https://www.thebangkokinsight.com/news/busines...,2022-08-18,2022-11-15
4,นายกฯ แฮปปี้ ส่งออก ‘ทุเรียนไทย’ ไปจีน ทะลุเป้...,นายกฯ แฮปปี้ ส่งออก ‘ทุเรียนไทย’ ไปจีน ทะลุเป้...,https://www.thebangkokinsight.com/news/busines...,2022-06-24,2022-11-15


In [9]:
df_json = df.to_dict('records')

In [10]:
print(f'Total news : {len(df_json)}')

Total news : 34


In [11]:
with open('exporting_durians_the_bangkok_insight.json', 'w', encoding='utf8') as f:
    json.dump(df_json, f)

## Upload data to DataPlatform

In [12]:
from dsmlibrary.datanode import DataNode

In [13]:
dir_raw_id = 60

In [14]:
datanode = DataNode()

Init DataNode sucessful!


In [ ]:
datanode.upload_file(directory_id=dir_raw_id, file_path='exporting_durians_the_bangkok_insight.json')